In [10]:
import pandas as pd
import numpy as np

from src.api import db
import torch

In [7]:
reviews = db.get_all_reviews()
reviews_df = pd.DataFrame(reviews)
books = db.get_all_books()
books_df = pd.DataFrame(books)
reg_users = db.get_all_registered_users()
reg_users = pd.DataFrame(reg_users)

ratings = reviews_df.drop_duplicates(subset=["user_id", "book_id"]).copy()
ratings_df = ratings.copy()
users = ratings["user_id"].unique()
books = ratings["book_id"].unique()

In [16]:
users

array(['A7IMBNFYANPNV', 'A1WV4Q44JE40UF', 'A32MYDPSMCHT1L', ...,
       'A1EB7BDZI63ILF', 'A1S6Q9I87YY21C', 'AZ1G140XCLW3I'], dtype=object)

In [13]:
reg_users["id"]

0    1
1    2
2    3
3    4
Name: id, dtype: int64

In [17]:
c = 0
for ru in reg_users["id"]:
    print(ru)
    if not ru in users:
        users = np.append(users,(ru))
        c += 1
c

1
2
3
4


4

In [18]:
users

array(['A7IMBNFYANPNV', 'A1WV4Q44JE40UF', 'A32MYDPSMCHT1L', ..., 2, 3, 4],
      dtype=object)

In [19]:

userid2idx = {o:i for i,o in enumerate(users)}
bookid2idx = {o:i for i,o in enumerate(books)}

idx2userid = {i:o for o,i in userid2idx.items()}
idx2bookid = {i:o for o,i in bookid2idx.items()}

users_items_pivot_matrix_df = ratings.pivot(
            index="user_id", 
            columns="book_id", 
            values="score").fillna(0)
        
X = torch.Tensor(users_items_pivot_matrix_df.to_numpy())

In [22]:
X_suppl = np.zeros((c, X.shape[1]))
X_suppl.shape

(4, 97)

In [24]:
np.concatenate([X, X_suppl]).shape

(33774, 97)

In [21]:
X.shape, len(users)

(torch.Size([33770, 97]), 33774)